In [1]:
import time
import os
import math
import numpy as np
import cv2
import pandas as pd
import pyautogui as pa
import shutil
import imageio

from lib.auto_GUI.auto_GUI_base import AutoGUIBase
from lib.auto_GUI.auto_DAT import AutoDAT
from lib.auto_GUI.auto_PhotoZ import AutoPhotoZ

In [2]:
# Make a series of 1-frame measure windows from PhotoZ

# Load data, not from Drive for Desktop since PhotoZ complains about loading zda from Drive
google_drive_dir = "G:/.shortcut-targets-by-id/1afwzLrN3AQQPbJ1PQiZlKAb1Bijc5qln/Choosing trajectories/"
data_dir = "C:/Users/jjudge3/Desktop/Data/movie_targets/"
filter_keyword = None  # None to process all, else a filter to search in file full path
start_frame = 94
movie_length = 27

disable_photoZ_interact = False
disable_google_drive_interact = True
initialize_photoZ = False
clean_scratch_files = False  # clean up SNR maps
replace_existing_frame_pngs = True

In [3]:
# set up PhotoZ (open it manually)
aPhz = AutoPhotoZ(data_dir=data_dir)
if initialize_photoZ and not disable_photoZ_interact:
    aPhz.prepare_photoZ()
if not disable_photoZ_interact:
    aPhz.click_normalize_2_array_max()
pa.alert("Please disable trace overlay and maximize PhotoZ before continuing.")

'OK'

In [5]:
current_color_bound_setting = 1.0
for subdir, dirs, files in os.walk(data_dir):
    if 'notUsable' in subdir:
        continue
    if 'mm_hidden' in subdir:
        continue
    if 'converted_zda' in subdir:
        continue
    
    zda_file = None
    # locate zda file
    for f in files:
        if f.endswith('.zda'):
            zda_file = f
        else:
            continue

        rec_id = zda_file.split('.')[0]
        print(rec_id)
        # movie dir
        movie_dir = subdir + "/movie" + rec_id + "/"
        try:
            os.makedirs(movie_dir)
        except Exception as e:
            pass

        # determine if we are even missing any jpg'd frames
        need_to_open_zda = True
        for frame in range(start_frame, start_frame + movie_length):
            filename = movie_dir + str(frame) + ".jpg"
            if not os.path.exists(filename):
                need_to_open_zda = True
                break
        if not need_to_open_zda and not replace_existing_frame_pngs:
            continue
            
        recording_max_snr = 20.0  # gets changed
        
        if not disable_photoZ_interact:
            aPhz = AutoPhotoZ(data_dir=subdir)
            # open the PhotoZ file
            aPhz.select_PhotoZ()

            print("\n\nOpening", zda_file)
            aPhz.open_zda_file(subdir + "/" + zda_file)

            # estimate the SNR max across all frames
            aPhz.set_measure_window(start_frame, movie_length)
            filename = aPhz.save_background()
            if filename is not None:  # otherwise, sticks with default rec max of 20
                
                # open Data.dat and read in the array max
                intermed_snr_df = pd.read_csv(filename, 
                                              sep='\t', 
                                              header=None, 
                                              names=['Index',  'Values'])
                recording_max_snr = intermed_snr_df['Values'].max()

                recording_max_snr *= 1.05  # 5% higher for headroom
                recording_max_snr = round(recording_max_snr, 2)
                if clean_scratch_files:
                    try:
                        os.remove(filename)
                    except Exception as e:
                        print("Could not remove", filename)
                        print(e)
            
            # set the normalization to be consistent across entire video
            aPhz.set_color_upper_bound(recording_max_snr, current_color_bound_setting)
            print("Moved color upper bound from", current_color_bound_setting, "to", recording_max_snr)
            current_color_bound_setting = recording_max_snr
            
            # set measure window width to 1
            aPhz.set_measure_window(None, 1)
        
        images = []
        img_filenames = []
        for frame in range(start_frame, start_frame + movie_length):
            filename = movie_dir + str(frame) + ".jpg"
            if replace_existing_frame_pngs or not os.path.exists(filename):
                # change frame
                if not disable_photoZ_interact:
                    aPhz.set_measure_window(frame, None)

                    # export this frame
                    aPhz.save_map_jpeg(filename)
                    print("File created:", filename)
                    
            
            try:
                images.append(imageio.imread(filename))
                img_filenames.append(filename)
            except Exception as e:
                pass
        
        # create gif
        created_movie = False
        try:
            imageio.mimsave(movie_dir + rec_id + 'movie.gif', images)
            print("CREATED MOVIE:", rec_id + 'movie.gif')
            created_movie = True
            if clean_scratch_files:
                for img_file in img_filenames:
                    os.remove(img_file)
        except Exception as e:
            if not created_movie:
                print("Not creating movie for " + rec_id)
            print(e)

04_01_01


Opening 04_01_01.zda
Moved color upper bound from 1.0 to 9.34
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/94.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/95.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/98.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/99.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/100.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/101.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/02-10-23\4_1/movie04_01_01/103.jpg
Fil

File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/96.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/97.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/98.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/99.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/100.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/101.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_02_05/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/

File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/98.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/99.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/100.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/101.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_03_06/107.jpg
File created: C:/Users/jjudge3/Desktop/Dat

File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/100.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/101.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie01_05_06/109.jpg
File created: C:/Users/jjudge3/Desktop/D

File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_02_02/111.jpg
File created: C:/Users/jjudge3/Desktop/D

File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/111.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/112.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/03-07-23_movies/movie02_03_04/113.jpg
File created: C:/Users/jjudge3/Desktop/D

File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/111.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/112.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\02_01_03/movie02_01_03/113.jpg


File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/109.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/110.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_01_01/movie03_01_01/111.jpg


File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/100.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/101.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/107.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/108.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\03_02_02/movie03_02_02/109.jpg


File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/98.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/99.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/100.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/101.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/102.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/103.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/104.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/105.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/106.jpg
File created: C:/Users/jjudge3/Desktop/Data/movie_targets/2023-02-28\06_01_02/movie06_01_02/107.jpg
Fi

In [ ]:
# copying movies into Google Drive
if not disable_google_drive_interact:
    for subdir, dirs, files in os.walk(data_dir):
        if 'notUsable' in subdir:
            continue
        if 'mm_hidden' in subdir:
            continue
            
        extend = subdir.split("movie_targets/")[-1]
        if "movie" in extend:
            
            dst_dir = google_drive_dir + extend
            if os.path.exists(dst_dir):
                try:
                    shutil.rmtree(dst_dir, ignore_errors=False, onerror=None)
                except Exception as e:
                    print("Dest dir", dst_dir, "not deleted. Skipping.")
                    print(e)
                    continue

            print(subdir, "\n", dst_dir, "\n\n")
            shutil.copytree(subdir, dst_dir)


In [ ]:
# clean up local drive
if not disable_google_drive_interact:
    for subdir, dirs, files in os.walk(data_dir):
        if 'notUsable' in subdir:
            continue

        if "movie" in subdir:
            try:
                shutil.rmtree(subdir, ignore_errors=False, onerror=None)
            except Exception as e:
                print("Dir", subdir, "not deleted. Skipping.")
                print(e)
                continue

            print("Deleted:", subdir)